# Proyek Analisis Sentimen: [Youtube-Comments-dataset]
- **Nama:** [Tsamarah Muthi'ah Abdullah]
- **Email:** [a135xaf486@devacademy.id]
- **ID Dicoding:** [a135xaf48]

# Import Library

In [ ]:
# Menginstall google API
!pip install --upgrade google-api-python-client pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 78.4 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.160.0
    Uninstalling google-api-python-client-2.160.0:
      Successfully uninstalled google-api-python-client-2.160.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [ ]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 11.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [ ]:
# Import Library
import googleapiclient.discovery
import pandas as pd
import time
import random
import re
import string
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Conv1D, GlobalMaxPooling1D, Bidirectional, BatchNormalization
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Bidirectional, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

# Data Preprocessing dan Pembersihan

In [ ]:
# Membaca dan Menampilkan informasi dataset
dataset = pd.read_csv("NDsO1LT_0lw_youtube_comments.csv")
print("Jumlah data:", len(dataset))

Jumlah data: 85028


In [ ]:
print(dataset.head())

             username                                               text  \
0            @MrBeast  BEAST GAMES FINALE DROPS FEBRUARY 13TH! GO WAT...   
1     @SultanRayyan-7                          Saya orang Indonesia 🇮🇩🇮🇩   
2  @MohamadrezaRezayy  مستر بیست عزیز ❤ لطفا دوبله فارسی این ویدیو را...   
3       @ILoveyou-954  বাংলাদেশ আসবেন কবে পাবনা জেলার কিছু দেখতে চাই ...   
4     @Moneymakerarab  This is for girls listen to Dalida - Helwa ya ...   

   likes          published_at  
0  54600  2025-02-08T16:59:31Z  
1      0  2025-03-19T21:51:42Z  
2      0  2025-03-19T21:43:03Z  
3      0  2025-03-19T21:34:32Z  
4      0  2025-03-19T21:20:21Z  


In [ ]:
print("Informasi Dataset:")
print(dataset.info())

Informasi Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85028 entries, 0 to 85027
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   username      84982 non-null  object
 1   text          85028 non-null  object
 2   likes         85028 non-null  int64 
 3   published_at  85028 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.6+ MB
None


In [ ]:
print('\nStatistik Deskriptif:')
print(dataset.describe())


Statistik Deskriptif:
               likes
count   85028.000000
mean        9.727654
std       744.702565
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max    167348.000000


In [ ]:
# Mengecek jumlah missing values di setiap kolom
print(dataset.isnull().sum())

username        46
text             0
likes            0
published_at     0
dtype: int64


In [ ]:
# Menghapus nilai kosong
dataset.dropna(inplace=True)

In [ ]:
# Menghapus duplikasi
dataset.drop_duplicates(subset=['text'], inplace=True)

In [ ]:
# Pembersihan teks
def clean_text(text):
    text = text.lower()
    text = ''.join(c for c in text if c.isalnum() or c.isspace())
    return text

dataset['cleaned_text'] = dataset['text'].apply(clean_text)
print("Data setelah pembersihan:")
print(dataset.head())

Data setelah pembersihan:
             username                                               text  \
0            @MrBeast  BEAST GAMES FINALE DROPS FEBRUARY 13TH! GO WAT...   
1     @SultanRayyan-7                          Saya orang Indonesia 🇮🇩🇮🇩   
2  @MohamadrezaRezayy  مستر بیست عزیز ❤ لطفا دوبله فارسی این ویدیو را...   
3       @ILoveyou-954  বাংলাদেশ আসবেন কবে পাবনা জেলার কিছু দেখতে চাই ...   
4     @Moneymakerarab  This is for girls listen to Dalida - Helwa ya ...   

   likes          published_at  \
0  54600  2025-02-08T16:59:31Z   
1      0  2025-03-19T21:51:42Z   
2      0  2025-03-19T21:43:03Z   
3      0  2025-03-19T21:34:32Z   
4      0  2025-03-19T21:20:21Z   

                                        cleaned_text  
0  beast games finale drops february 13th go watc...  
1                              saya orang indonesia   
2  مستر بیست عزیز  لطفا دوبله فارسی این ویدیو را ...  
3                    বলদশ আসবন কব পবন জলর কছ দখত চই   
4  this is for girls listen to dalida

In [ ]:
# Mengecek kembali jumlah missing values di setiap kolom
print(dataset.isnull().sum())

username        0
text            0
likes           0
published_at    0
cleaned_text    0
dtype: int64


# SKEMA 1

# **2. Ekstraksi Fitur dan Pelabelan Data**

Metode yang digunakan bebas sesuai dengan preferensi masing-masing peserta. Tahapan ini penting untuk mempersiapkan data sehingga dapat diolah lebih lanjut dalam proses pelatihan model.

In [ ]:
# Melabeli komentar secara manual (positif, netral, negatif)
def label_comment(text):
    if any(word in text for word in ["love", "great", "awesome", "good", "nice", "amazing", "fantastic"]):
        return "positif"
    elif any(word in text for word in ["hate", "bad", "awful", "worst", "terrible", "disgusting", "boring"]):
        return "negatif"
    else:
        return "netral"

dataset['label'] = dataset['cleaned_text'].apply(label_comment)
print("Distribusi label:")
print(dataset['label'].value_counts())

Distribusi label:
label
netral     63766
positif     4728
negatif       69
Name: count, dtype: int64


In [ ]:
# Encoding label
label_encoder = LabelEncoder()
dataset['label_encoded'] = label_encoder.fit_transform(dataset['label'])

# **3. Pembangunan Model Deep Learning**

Pilihan algoritma pelatihan ini haruslah sesuai dengan tujuan analisis sentimen yang ingin dicapai.

# Skema 1: LSTM dengan Tokenizer + Padding (80/20)

**Algoritma:** Long Short-Term Memory (LSTM)

**Ekstraksi Fitur:** Tokenizer + Padding

**Alasan Pemilihan:**
- LSTM sangat efektif dalam memproses data berbasis urutan, seperti teks atau kalimat.
- Algoritma ini cocok untuk menangani long-term dependencies dalam urutan kata.
- Tokenizer digunakan untuk mengonversi kata menjadi indeks numerik.
- Padding digunakan untuk memastikan panjang urutan tetap sama agar dapat diproses dalam batch.

In [ ]:
# Tokenisasi teks dengan maksimal 10.000 kata
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(dataset['cleaned_text'])

In [ ]:
# Mengubah teks menjadi urutan token
sequences = tokenizer.texts_to_sequences(dataset['cleaned_text'])

In [ ]:
# Padding urutan agar memiliki panjang yang sama
padded = pad_sequences(sequences, maxlen=100, padding='post')

In [ ]:
# Mengatasi Ketidakseimbangan Data dengan SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(padded, dataset['label_encoded'])

In [ ]:
# Membagi data menjadi data latih dan data uji (80/20)
X_train, X_test, y_train, y_test = train_test_split(padded, dataset['label_encoded'], test_size=0.2, random_state=42)

In [ ]:
# Membangun Model
print("\nSkema 1: LSTM dengan Tokenizer + Padding (80/20)")

class_weights = {0: 1000, 1: 1, 2: 10}

model1 = Sequential([
    Embedding(10000, 128, input_length=100),
    Bidirectional(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),
    GlobalMaxPooling1D(),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')
])


Skema 1: LSTM dengan Tokenizer + Padding (80/20)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# Mengompilasi model dengan optimizer Adam
model1.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

In [ ]:
# Melatih model dengan data latih
model1.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=64, class_weight=class_weights)

Epoch 1/30
858/858 ━━━━━━━━━━━━━━━━━━━━ 621s 711ms/step - accuracy: 0.7042 - loss: 3.0077 - val_accuracy: 0.6939 - val_loss: 0.9136
Epoch 2/30
858/858 ━━━━━━━━━━━━━━━━━━━━ 637s 727ms/step - accuracy: 0.7162 - loss: 2.0105 - val_accuracy: 0.6921 - val_loss: 0.6790
Epoch 3/30
858/858 ━━━━━━━━━━━━━━━━━━━━ 618s 720ms/step - accuracy: 0.6885 - loss: 1.5884 - val_accuracy: 0.6890 - val_loss: 0.5883
Epoch 4/30
858/858 ━━━━━━━━━━━━━━━━━━━━ 626s 725ms/step - accuracy: 0.7220 - loss: 1.2107 - val_accuracy: 0.7147 - val_loss: 0.5296
Epoch 5/30
858/858 ━━━━━━━━━━━━━━━━━━━━ 624s 727ms/step - accuracy: 0.7228 - loss: 1.1010 - val_accuracy: 0.7704 - val_loss: 0.4286
Epoch 6/30
858/858 ━━━━━━━━━━━━━━━━━━━━ 678s 722ms/step - accuracy: 0.7777 - loss: 0.8334 - val_accuracy: 0.8156 - val_loss: 0.3570
Epoch 7/30
858/858 ━━━━━━━━━━━━━━━━━━━━ 622s 721ms/step - accuracy: 0.8497 - loss: 0.6095 - val_accuracy: 0.9255 - val_loss: 0.1743
Epoch 8/30
858/858 ━━━━━━━━━━━━━━━━━━━━ 619s 722ms/step - accuracy: 0.9092 -

In [ ]:
# Evaluasi Model Skema 1
print("\nEvaluasi Model Skema 1: LSTM dengan Tokenizer + Padding (80/20)")

# Prediksi pada data uji
y_pred1 = np.argmax(model1.predict(X_test), axis=1)

# Laporan Klasifikasi
print("\nClassification Report:")
print(classification_report(y_test, y_pred1, target_names=label_encoder.classes_))

# Akurasi
accuracy = accuracy_score(y_test, y_pred1)
print(f"Akurasi Model: {accuracy:.4f}")


Evaluasi Model Skema 1: LSTM dengan Tokenizer + Padding (80/20)
429/429 ━━━━━━━━━━━━━━━━━━━━ 40s 92ms/step

Classification Report:
              precision    recall  f1-score   support

     negatif       0.01      0.60      0.01        10
      netral       1.00      0.93      0.97     12792
     positif       0.97      0.97      0.97       911

    accuracy                           0.94     13713
   macro avg       0.66      0.84      0.65     13713
weighted avg       1.00      0.94      0.97     13713

Akurasi Model: 0.9372


In [ ]:
# Inference dan Testing Skema 1
print("\nInference Skema 1: LSTM dengan Tokenizer + Padding (80/20)")
# Fungsi Prediksi Teks
def predict_text_lstm(text, tokenizer, model1, label_encoder):
    # Tokenisasi dan padding teks
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=100, padding='post')

    # Prediksi sentimen
    prediction = model1.predict(padded_sequence)
    label_index = np.argmax(prediction, axis=1)[0]
    label = label_encoder.inverse_transform([label_index])[0]

    print(f"Teks: {text}")
    print(f"Prediksi Sentimen: {label}")

# Prediksi Teks Baru
print("\nPrediksi Teks Baru:")
predict_text_lstm("I love your video!", tokenizer, model1, label_encoder)


Inference Skema 1: LSTM dengan Tokenizer + Padding (80/20)

Prediksi Teks Baru:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Teks: I love your video!
Prediksi Sentimen: positif


**Insight Skema 1:**
1. Model ini memiliki akurasi yang tinggi, mencapai 93.72%. Hal ini menunjukkan bahwa model dapat mengklasifikasikan sebagian besar teks dengan benar.
2. Precision untuk kelas "negatif" sangat rendah (0.01) meskipun recall cukup tinggi (0.60). Ini menunjukkan bahwa model sering salah mengklasifikasikan teks negatif, mungkin karena data negatif sangat sedikit atau model terlalu fokus pada kelas "netral" dan "positif".
3. Precision, recall, dan f1-score pada kelas "netral" dan "positif" sangat tinggi (mendekati 1.00). Hal ini menunjukkan bahwa model mampu mengenali kedua sentimen tersebut dengan sangat baik.
4. Model masih memiliki kelemahan pada kelas negatif, sehingga peningkatan dapat dilakukan dengan teknik augmentasi atau modifikasi arsitektur.
5. Pada teks uji "I love your video!", model berhasil mengklasifikasikannya sebagai "positif", yang merupakan prediksi yang tepat.


# SKEMA 2

# **2. Ekstraksi Fitur dan Pelabelan Data**

Metode yang digunakan bebas sesuai dengan preferensi masing-masing peserta. Tahapan ini penting untuk mempersiapkan data sehingga dapat diolah lebih lanjut dalam proses pelatihan model.

In [ ]:
# Melabeli komentar secara manual (positif, netral, negatif)
def label_comment(text):
    if any(word in text for word in ["love", "great", "awesome", "good", "nice", "amazing", "fantastic"]):
        return "positif"
    elif any(word in text for word in ["hate", "bad", "awful", "worst", "terrible", "disgusting", "boring"]):
        return "negatif"
    else:
        return "netral"

dataset['label'] = dataset['cleaned_text'].apply(label_comment)
print("Distribusi label:")
print(dataset['label'].value_counts())

Distribusi label:
label
netral     63770
positif     4724
negatif       69
Name: count, dtype: int64


In [ ]:
# Encoding label
label_encoder = LabelEncoder()
dataset['label_encoded'] = label_encoder.fit_transform(dataset['label'])

# One-Hot Encoding pada label
y = tf.keras.utils.to_categorical(dataset['label_encoded'], num_classes=len(label_encoder.classes_))

print("Label Setelah Encoding:")
print(y[:5])
print("Classes:", label_encoder.classes_)

Label Setelah Encoding:
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
Classes: ['negatif' 'netral' 'positif']


In [ ]:
# Tokenizer dan Padding
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(dataset['cleaned_text'])
sequences = tokenizer.texts_to_sequences(dataset['cleaned_text'])
padded = pad_sequences(sequences, maxlen=100, padding='post')

In [ ]:
# Mengatasi Ketidakseimbangan Data dengan SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(padded, dataset['label_encoded'])

# **3. Pembangunan Model Deep Learning**

Pilihan algoritma pelatihan ini haruslah sesuai dengan tujuan analisis sentimen yang ingin dicapai.

# Skema 2: CNN dengan Word2Vec (80/20)

**Algoritma:** Convolutional Neural Network (CNN)

**Ekstraksi Fitur:** Word2Vec

**Alasan Pemilihan:**
- CNN dikenal dalam pengenalan pola spasial, dan bisa dimanfaatkan untuk menangkap pola lokal pada teks (misalnya n-grams).
- Word2Vec menghasilkan representasi vektor dari kata-kata dengan mempertimbangkan hubungan semantik dan sintaksis.
- Kombinasi CNN + Word2Vec memungkinkan model mempelajari fitur spasial dari representasi kata yang lebih kaya.

In [ ]:
# Membagi data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [ ]:
# Membuat model Word2Vec menggunakan teks bersih
word2vec = Word2Vec(sentences=[text.split() for text in dataset['cleaned_text']], vector_size=100, window=5, min_count=1)

In [ ]:
# Menyiapkan embedding matrix
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 100))
for word, i in tokenizer.word_index.items():
    if word in word2vec.wv:
        embedding_matrix[i] = word2vec.wv[word]

In [ ]:
# Membangun model CNN
model2 = Sequential([
    Embedding(len(tokenizer.word_index) + 1, 100, weights=[embedding_matrix], input_length=100, trainable=False),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model2.fit(X_train, y_train, epochs=80, validation_data=(X_test, y_test), batch_size=64)

Epoch 1/80
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 116s 46ms/step - accuracy: 0.7494 - loss: 0.6190 - val_accuracy: 0.8440 - val_loss: 0.3897
Epoch 2/80
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 142s 46ms/step - accuracy: 0.8359 - loss: 0.4196 - val_accuracy: 0.8638 - val_loss: 0.3493
Epoch 3/80
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 157s 52ms/step - accuracy: 0.8556 - loss: 0.3729 - val_accuracy: 0.8774 - val_loss: 0.3172
Epoch 4/80
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 132s 48ms/step - accuracy: 0.8664 - loss: 0.3488 - val_accuracy: 0.8786 - val_loss: 0.3077
Epoch 5/80
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 137s 46ms/step - accuracy: 0.8774 - loss: 0.3228 - val_accuracy: 0.8881 - val_loss: 0.2925
Epoch 6/80
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 154s 51ms/step - accuracy: 0.8827 - loss: 0.3106 - val_accuracy: 0.8799 - val_loss: 0.2979
Epoch 7/80
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 134s 48ms/step - accuracy: 0.8866 - loss: 0.3024 - val_accuracy: 0.8938 - val_loss: 0.2808
Epoch 8/80
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 112s 47ms/step - accuracy: 

In [ ]:
# Evaluasi model pada data testing
# Prediksi pada data uji
y_pred1 = np.argmax(model2.predict(X_test), axis=1)

# Laporan Klasifikasi
print("\nClassification Report:")
print(classification_report(y_test, y_pred1, target_names=label_encoder.classes_))

# Akurasi
accuracy = accuracy_score(y_test, y_pred1)
print(f"Akurasi Model: {accuracy:.4f}")

1196/1196 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step

Classification Report:
              precision    recall  f1-score   support

     negatif       0.89      0.93      0.91     12770
      netral       0.92      0.96      0.94     12699
     positif       0.95      0.85      0.90     12793

    accuracy                           0.92     38262
   macro avg       0.92      0.92      0.92     38262
weighted avg       0.92      0.92      0.92     38262

Akurasi Model: 0.9168


In [ ]:
# Inference dan Testing Skema 2
print("\nInference Skema 2: CNN dengan Word2Vec (80/20)")
# Fungsi Prediksi Teks
def predict_text(text, tokenizer, model2, label_encoder):
    # Tokenisasi dan padding teks
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=100, padding='post')

    # Prediksi sentimen
    prediction = model2.predict(padded_sequence)
    label_index = np.argmax(prediction, axis=1)[0]
    label = label_encoder.inverse_transform([label_index])[0]

    print(f"Teks: {text}")
    print(f"Prediksi Sentimen: {label}")

# Prediksi Teks Baru
print("\nPrediksi Teks Baru:")
predict_text("As an egyptian, I didn't know about the Sphinx.", tokenizer, model2, label_encoder)


Inference Skema 2: CNN dengan Word2Vec (80/20)

Prediksi Teks Baru:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Teks: As an egyptian, I didn't know about the Sphinx.
Prediksi Sentimen: netral


**Insight Skema 2 :**

1. Model ini memiliki akurasi yang sangat baik, yaitu 91.68%, menunjukkan kemampuan model dalam mengklasifikasikan sentimen dengan cukup akurat.

2. Kinerja Model pada Setiap Kelas

- Negatif: Precision 0.89, Recall 0.93, F1-Score 0.91
- Netral: Precision 0.92, Recall 0.96, F1-Score 0.94
- Positif: Precision 0.95, Recall 0.85, F1-Score 0.90
- Model memiliki keseimbangan performa yang baik di ketiga kelas, dengan sedikit penurunan pada recall kelas "positif".
3. Pada teks "As an Egyptian, I didn't know about the Sphinx.", model memprediksi sentimen sebagai "netral", yang memang sesuai dengan konteks kalimat.

# SKEMA 3

# **2. Ekstraksi Fitur dan Pelabelan Data**

Metode yang digunakan bebas sesuai dengan preferensi masing-masing peserta. Tahapan ini penting untuk mempersiapkan data sehingga dapat diolah lebih lanjut dalam proses pelatihan model.

In [ ]:
# Melabeli komentar secara manual (positif, netral, negatif)
def label_comment(text):
    if any(word in text for word in ["love", "great", "awesome", "good", "nice", "amazing", "fantastic"]):
        return "positif"
    elif any(word in text for word in ["hate", "bad", "awful", "worst", "terrible", "disgusting", "boring"]):
        return "negatif"
    else:
        return "netral"

dataset['label'] = dataset['cleaned_text'].apply(label_comment)
print("Distribusi label:")
print(dataset['label'].value_counts())

Distribusi label:
label
netral     63770
positif     4724
negatif       69
Name: count, dtype: int64


In [ ]:
# Encode label dengan LabelEncoder
le = LabelEncoder()
dataset['label'] = le.fit_transform(dataset['label'])

# **3. Pembangunan Model Deep Learning**

Pilihan algoritma pelatihan ini haruslah sesuai dengan tujuan analisis sentimen yang ingin dicapai.

# Skema 3: GRU dengan Word2Vec (70/30)

**Algoritma:** GRU (Gated Recurrent Unit)

**Ekstraksi Fitur:** Word2Vec

**Alasan Pemilihan:**

- GRU lebih sederhana dan lebih ringan dibandingkan dengan LSTM, sehingga lebih cepat dalam proses pelatihan tanpa kehilangan banyak informasi temporal. Cocok untuk dataset teks dengan jumlah data besar.
- Word2Vec digunakan untuk menghasilkan vektor kata yang memiliki makna semantik sehingga model dapat memahami konteks dari kata-kata dalam kalimat.

In [ ]:
print("\nSkema 3: GRU dengan Word2Vec (70/30)")
# Split data 70/30
X = dataset['cleaned_text']
y = dataset['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


Skema 3: GRU dengan Word2Vec (70/30)


In [ ]:
# Tokenizer dan Padding
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)  # Hanya fit pada data latih untuk mencegah data bocor
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Padding
X_train_padded = pad_sequences(X_train_sequences, maxlen=209, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=209, padding='post')

In [ ]:
# Mengatasi Ketidakseimbangan Data dengan SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_padded, y_train)

print("Distribusi label setelah SMOTE:")
print(pd.Series(y_resampled).value_counts())

Distribusi label setelah SMOTE:
label
1    44619
2    44619
0    44619
Name: count, dtype: int64


In [ ]:
# One-Hot Encoding pada label
encoder = LabelEncoder()
y_resampled_encoded = encoder.fit_transform(y_resampled)  # Transform label SMOTE
y_test_encoded = encoder.transform(y_test)  # Transform label uji


In [ ]:
# One-Hot Encoding
y_resampled_encoded = tf.keras.utils.to_categorical(y_resampled_encoded, num_classes=3)
y_test_encoded = tf.keras.utils.to_categorical(y_test_encoded, num_classes=3)

print("Label Setelah One-Hot Encoding:")
print(y_resampled_encoded[:5])
print("Classes:", encoder.classes_)

Label Setelah One-Hot Encoding:
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]
Classes: [0 1 2]


In [ ]:
# Model GRU
model3 = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=209),  # Embedding layer lebih kecil
    Bidirectional(GRU(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),  # GRU lebih kecil
    GlobalMaxPooling1D(),  # Pooling
    BatchNormalization(),  # Normalisasi batch
    Dropout(0.3),  # Dropout lebih rendah
    Dense(64, activation='relu'),  # Layer Dense lebih kecil
    Dropout(0.3),  # Dropout lebih rendah
    Dense(32, activation='relu'),  # Layer Dense lebih kecil
    Dropout(0.3),  # Dropout lagi
    Dense(3, activation='softmax')  # Output layer dengan 3 kelas
])

In [ ]:
model3.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Melatih model dengan data seimbang
history = model3.fit(
    X_resampled, y_resampled_encoded,  # Data dan label sudah One-Hot
    epochs=10,
    validation_data=(X_test_padded, y_test_encoded),  # Data validasi juga One-Hot
    batch_size=64
)

Epoch 1/10
2092/2092 ━━━━━━━━━━━━━━━━━━━━ 1219s 583ms/step - accuracy: 0.9227 - loss: 0.2071 - val_accuracy: 0.9725 - val_loss: 0.1052
Epoch 2/10
2092/2092 ━━━━━━━━━━━━━━━━━━━━ 1227s 585ms/step - accuracy: 0.9344 - loss: 0.1814 - val_accuracy: 0.9698 - val_loss: 0.1081
Epoch 3/10
2092/2092 ━━━━━━━━━━━━━━━━━━━━ 1205s 576ms/step - accuracy: 0.9424 - loss: 0.1597 - val_accuracy: 0.9700 - val_loss: 0.1204
Epoch 4/10
2092/2092 ━━━━━━━━━━━━━━━━━━━━ 1202s 575ms/step - accuracy: 0.9476 - loss: 0.1458 - val_accuracy: 0.9578 - val_loss: 0.1437
Epoch 5/10
2092/2092 ━━━━━━━━━━━━━━━━━━━━ 1219s 573ms/step - accuracy: 0.9534 - loss: 0.1327 - val_accuracy: 0.9736 - val_loss: 0.1010
Epoch 6/10
2092/2092 ━━━━━━━━━━━━━━━━━━━━ 1254s 589ms/step - accuracy: 0.9583 - loss: 0.1194 - val_accuracy: 0.9632 - val_loss: 0.1492
Epoch 7/10
2092/2092 ━━━━━━━━━━━━━━━━━━━━ 1259s 578ms/step - accuracy: 0.9602 - loss: 0.1150 - val_accuracy: 0.9736 - val_loss: 0.1047
Epoch 8/10
2092/2092 ━━━━━━━━━━━━━━━━━━━━ 1197s 566ms/s

In [ ]:
# Preprocessing data testing
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=209, padding='post')

# Evaluasi model pada data testing
test_loss, test_accuracy = model3.evaluate(X_test_padded, y_test_encoded)
print(f"Akurasi Testing (dari model.evaluate): {test_accuracy:.2%}")

643/643 ━━━━━━━━━━━━━━━━━━━━ 52s 80ms/step - accuracy: 0.9723 - loss: 0.1182
Akurasi Testing (dari model.evaluate): 97.05%


In [ ]:
target_names = [str(cls) for cls in le.classes_]
print(target_names)  # Pastikan hasilnya dalam bentuk list of strings

['0', '1', '2']


In [ ]:
print(type(le.classes_))
print(le.classes_)
print(list(le.classes_))

<class 'numpy.ndarray'>
[0 1 2]
[np.int64(0), np.int64(1), np.int64(2)]


In [ ]:
# Prediksi pada data testing
y_pred2 = np.argmax(model3.predict(X_test_padded), axis=1)
y_true = np.argmax(y_test_encoded, axis=1)

print(classification_report(y_true, y_pred2, target_names=target_names))

643/643 ━━━━━━━━━━━━━━━━━━━━ 52s 81ms/step
              precision    recall  f1-score   support

           0       0.02      0.42      0.04        12
           1       1.00      0.97      0.98     19151
           2       0.80      0.97      0.87      1406

    accuracy                           0.97     20569
   macro avg       0.61      0.78      0.63     20569
weighted avg       0.98      0.97      0.98     20569



In [ ]:
# Inference  Skema 3
print("\nInference Skema 3: GRU dengan Word2Vec (70/30)")
def predict_text(text, tokenizer, model3, le, max_length=100):
    # Preprocessing teks
    text = text.lower()
    text_seq = tokenizer.texts_to_sequences([text])
    text_pad = pad_sequences(text_seq, maxlen=max_length, padding='post')

    # Prediksi
    prediction = model3.predict(text_pad)
    predicted_label = np.argmax(prediction, axis=1)[0]
    predicted_class = le.inverse_transform([predicted_label])[0]

    print(f"Teks: {text}")
    print(f"Prediksi Label: {predicted_class}")
    print(f"Confidence: {np.max(prediction) * 100:.2f}%")
    return predicted_class


Inference Skema 3: GRU dengan Word2Vec (70/30)


In [ ]:
# Melakukan Prediksi pada Teks Baru (0: Negatif, 1: Netral, 2: Positif)
predict_text("I love how you guys taking the video, that was amazing!", tokenizer, model3, le)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Teks: i love how you guys taking the video, that was amazing!
Prediksi Label: 2
Confidence: 100.00%


np.int64(2)

**Insight Skema 3:**

1. Model memiliki akurasi tinggi sebesar 97.05%, yang menunjukkan bahwa model secara keseluruhan cukup andal dalam mengklasifikasikan data.
2. **Kelas 0 (Negatif):**

- Precision: 0.02 (Sangat rendah)

- Recall: 0.42 (Cukup tinggi untuk jumlah data yang sedikit)

- F1-Score: 0.04 (Buruk)

- Jumlah data: 12 (Sangat sedikit)

- **Catatan:** Model mengalami kesulitan dalam mengklasifikasikan kelas ini karena jumlah sampel yang sangat sedikit sehingga model cenderung mengabaikannya.

3. **Kelas 1 (Netral):**

- Precision: 1.00 (Sangat baik)

- Recall: 0.97 (Sangat baik)

- F1-Score: 0.98 (Sangat tinggi)

- Jumlah data: 19,151 (Mayoritas data)

- **Catatan:** Model sangat akurat dalam mendeteksi kelas ini karena data yang melimpah

4. **Kelas 2 (Positif):**

- Precision: 0.80 (Baik)

- Recall: 0.97 (Sangat baik)

- F1-Score: 0.87 (Tinggi)

- Jumlah data: 1,406 (Cukup banyak)

- **Catatan**: Model juga mampu menangani kelas positif dengan cukup baik.

5. Model memprediksi teks "i love how you guys taking the video, that was amazing!" sebagai kelas 2 (Positif) dengan 100% confidence.